In [1]:
# set the environment variables needed for openai package to know to reach out to azure
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://<your-endpoint.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "your AzureOpenAI key"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

In [7]:
text = "This is a test document."

query_result = embeddings.embed_query(text)

[-0.003158408682793379,
 0.011094409972429276,
 -0.0040013170801103115,
 -0.011747414246201515,
 -0.0010153218172490597,
 0.010781234130263329,
 -0.010368109680712223,
 -0.005297331139445305,
 -0.009881687350571156,
 -0.02616015262901783,
 0.020376399159431458,
 0.022575292736291885,
 -0.007522876374423504,
 0.01728462427854538,
 -0.006003641523420811,
 0.01912369765341282,
 0.02125595696270466,
 -0.015645449981093407,
 0.007669468875974417,
 -0.018364081159234047,
 -0.0006909018848091364,
 -0.006416766904294491,
 -0.01098113413900137,
 0.017950955778360367,
 -0.02213551290333271,
 -0.003076783148571849,
 0.01437942124903202,
 -0.029984891414642334,
 0.01852400042116642,
 -0.007916011847555637,
 0.010068260133266449,
 -0.019456863403320312,
 -0.003688141703605652,
 -0.024241119623184204,
 -0.005530546884983778,
 0.003824739484116435,
 -0.005653818137943745,
 -0.029451826587319374,
 0.018603960052132607,
 -0.016618292778730392,
 0.00752953952178359,
 0.012420408427715302,
 -0.0009903343

In [25]:
doc_result = embeddings.embed_documents([text, text, text])

In [3]:
from langchain.vectorstores.redis import Redis
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./contract.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=20)
docs = text_splitter.split_documents(documents)



In [4]:
rds = Redis.from_documents(
    docs,
    embeddings,
    redis_url="redis://localhost:6379",
    index_name="link",
)

In [5]:
rds.similarity_search("在什么情况下可以单方面解除合同")

[Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除的； 2．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作，甲方单方面解除本合同的； \n3．乙方年度考核或者聘期考核不合格， 又不同意甲方调整其工作岗位的，', metadata={'source': './contract.pdf', 'page': 7}),
 Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除的； 2．乙方患病或者非因工负伤，医疗期满 后，不能从事原工作也不能从事由\n甲方安排的其他工作，甲方单方面解除本合同的； \n3．乙方年度考核或者聘期考核不合格， 又不同意甲方调整其工作岗位的，', metadata={'source': './contract.pdf', 'page': 7}),
 Document(page_content='（二）乙方有下列情形之一的，甲方可以随时单方面解除本合同： 1．在试用期内被证明不符合本岗位要求； 2．连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的； 3．乙方公派出国或因私出境逾期不归；   \n4．违反工作规定或者操作规程，发生责 任事故，或者失职、渎职，造成严\n重后果的； \n5．严重扰

In [ ]:
retriever = rds.as_retriever()

In [ ]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever